In [1]:
import os
import gradio as gr
from swarmauri.standard.llms.concrete.GroqModel import GroqModel
from swarmauri.standard.conversations.concrete.MaxSystemContextConversation import MaxSystemContextConversation
from swarmauri.standard.conversations.concrete.Conversation import Conversation
from swarmauri.standard.vector_stores.concrete.TfidfVectorStore import TfidfVectorStore
from swarmauri.standard.messages.concrete.SystemMessage import SystemMessage
from swarmauri.standard.messages.concrete.HumanMessage import HumanMessage
from swarmauri.standard.documents.concrete.Document import Document
from swarmauri.standard.agents.concrete.RagAgent import RagAgent
from swarmauri.standard.agents.concrete.SimpleConversationAgent import SimpleConversationAgent


os.environ['GROQ_API_KEY'] = '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'
llm = GroqModel(api_key=os.environ['GROQ_API_KEY'])

In [2]:
def extract_text_from_pdf(pdf_file_path):
    import fitz
    document = fitz.open(pdf_file_path)
    text = ""
    for page_num in range(document.page_count):
        page = document[page_num]
        text += page.get_text()
    document.close()
    return text

In [3]:
def process_pdf(input_text, history, pdf_file):
    if pdf_file:
        temp_path = pdf_file.name
        text = extract_text_from_pdf(temp_path)

        system_context = SystemMessage(content='You are a RAG assistant that gives information about the pdf uploaded.')
        conversation = MaxSystemContextConversation(system_context=system_context)
        vector_store = TfidfVectorStore()
        documents = [Document(content=text)]
        vector_store.add_documents(documents)

        agent = RagAgent(
            llm=llm,
            conversation=conversation,
            system_context=system_context,
            vector_store=vector_store
        )
    else:
#         system_context = SystemMessage(content='You are a good conversation partner.')
        conversation = Conversation()
        initial_user_message = HumanMessage(content="Hello")
        conversation.add_message(initial_user_message)
        
        agent = SimpleConversationAgent(
            llm=llm,
            conversation=conversation
        )
    
    response = agent.exec(input_text)
    return str(response)

In [4]:
def main():
    iface = gr.ChatInterface(
        fn=process_pdf,
        additional_inputs=gr.File(label="Upload PDF"),
        title="PDF Document Analyzer",
        description="Upload a PDF file to extract text and analyze it."
    )
    iface.launch(share=True)

if __name__ == "__main__":
    main()

Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://235b8f1f6e2ed5bbd2.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
